# DAT565 Introduction to Data Science and AI
## 2023-2024, LP1
## Assignment 5: Reinforcement Learning and Classification

The exercise takes place in a notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical.

The exercise takes place in this notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical.

*Tips:*
* You can execute certain Linux shell commands by prefixing the command with a `!`.
* You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results, the second you can use to write code snippets that execute the tasks required.  

This assignment is about **sequential decision making** under uncertainty (reinforcement learning). In a sequential decision process, the process jumps between different states (the *environment*), and in each state the decision maker, or *agent*, chooses among a set of actions. Given the state and the chosen action, the process jumps to a new state. At each jump the decision maker receives a reward, and the objective is to find a sequence of decisions (or an optimal *policy*) that maximizes the accumulated rewards.

We will use **Markov decision processes** (MDPs) to model the environment, and below is a primer on the relevant background theory.



* To make things concrete, we will first focus on decision making under **no** uncertainity (questions 1 and 2), i.e, given we have a world model, we can calculate the exact and optimal actions to take in it. We will first introduce **Markov Decision Process (MDP)** as the world model. Then we give one algorithm (out of many) to solve it.

* (optional) Next we will work through one type of reinforcement learning algorithm called Q-learning (question 3). Q-learning is an algorithm for making decisions under uncertainity, where uncertainity is over the possible world model (here MDP). It will find the optimal policy for the **unknown** MDP, assuming we do infinite exploration.

* Finally, in question 4 you will be asked to explain differences between reinforcement learning and supervised learning and in question 5 write about decision trees and random forests.

## Primer
### Decision Making
The problem of **decision making under uncertainty** (commonly known as **reinforcement learning**) can be broken down into
two parts. First, how do we learn about the world? This involves both the
problem of modeling our initial uncertainty about the world, and that of drawing conclusions from evidence and our initial belief. Secondly, given what we
currently know about the world, how should we decide what to do, taking into
account future events and observations that may change our conclusions?
Typically, this will involve creating long-term plans covering possible future
eventualities. That is, when planning under uncertainty, we also need to take
into account what possible future knowledge could be generated when implementing our plans. Intuitively, executing plans which involve trying out new
things should give more information, but it is hard to tell whether this information will be beneficial. The choice between doing something which is already
known to produce good results and experiment with something new is known
as the **exploration-exploitation dilemma**.

### The exploration-exploitation trade-off

Consider the problem of selecting a restaurant to go to during a vacation. Lets say the
best restaurant you have found so far was **Les Epinards**. The food there is
usually to your taste and satisfactory. However, a well-known recommendations
website suggests that **King’s Arm** is really good! It is tempting to try it out. But
there is a risk involved. It may turn out to be much worse than **Les Epinards**,
in which case you will regret going there. On the other hand, it could also be
much better. What should you do?
It all depends on how much information you have about either restaurant,
and how many more days you’ll stay in town. If this is your last day, then it’s
probably a better idea to go to **Les Epinards**, unless you are expecting **King’s
Arm** to be significantly better. However, if you are going to stay there longer,
trying out **King’s Arm** is a good bet. If you are lucky, you will be getting much
better food for the remaining time, while otherwise you will have missed only
one good meal out of many, making the potential risk quite small.

### Markov Decision Processes
Markov Decision Processes (MDPs) provide a mathematical framework for modeling sequential decision making under uncertainty. An *agent* moves between *states* in a *state space* choosing *actions* that affects the transition probabilities between states, and the subsequent *rewards* recieved after a jump. This is then repeated a finite or infinite number of epochs. The objective, or the *solution* of the MDP, is to optimize the accumulated rewards of the process.

Thus, an MDP consists of five parts:

* Decision epochs: $t={1,2,...,T}$, where $T\leq \infty$
* State space: $S=\{s_1,s_2,...,s_N\}$ of the underlying environment
* Action space $A=\{a_1,a_2,...,a_K\}$ available to the decision maker at each decision epoch
* Transition probabilities $p(s_{t+1}|s_t,a_t)$ for jumping from state $s_t$ to state $s_{t+1}$ after taking action $a_t$
* Reward functions $R_t = r(a_t,s_t,s_{t+1})$ resulting from the chosen action and subsequent transition

A *decision policy* is a function $\pi: s \rightarrow a$, that gives instructions on what action to choose in each state. A policy can either be *deterministic*, meaning that the action is given for each state, or *randomized* meaning that there is a probability distribution over the set of possible actions for each state. Given a specific policy $\pi$ we can then compute the the *expected total reward* when starting in a given state $s_1 \in S$, which is also known as the *value* for that state,

$$V^\pi (s_1) = E\left[ \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) {\Large |} s_1\right] = \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t)$$

where $a_t = \pi(s_t)$. To ensure convergence and to control how much credit to give to future rewards, it is common to introduce a *discount factor* $\gamma \in [0,1]$. For instance, if we think all future rewards should count equally, we would use $\gamma = 1$, while if we value near-future rewards higher than more distant rewards, we would use $\gamma < 1$. The expected total *discounted* reward then becomes

$$V^\pi( s_1) = \sum_{t=1}^T \gamma^{t-1} r(s_t,a_t, s_{t+1}) p(s_{t+1} | s_t, a_t) $$

Now, to find the *optimal* policy we want to find the policy $\pi^*$ that gives the highest total reward $V^*(s)$ for all $s\in S$. That is, we want to find the policy where

$$V^*(s) \geq V^\pi(s), s\in S$$

To solve this we use a dynamic programming equation called the *Bellman equation*, given by

$$V(s) = \max_{a\in A} \left\{\sum_{s'\in S} p(s'|s,a)( r(s,a,s') +\gamma V(s')) \right\}$$

It can be shown that if $\pi$ is a policy such that $V^\pi$ fulfills the Bellman equation, then $\pi$ is an optimal policy.

A real world example would be an inventory control system. The states could be the amount of items we have in stock, and the actions would be the amount of items to order at the end of each month. The discrete time would be each month and the reward would be the profit.


## Question 1

The first question covers a deterministic MPD, where the action is directly given by the state, described as follows:

* The agent starts in state **S** (see table below)
* The actions possible are **N** (north), **S** (south), **E** (east), and **W** west.
* The transition probabilities in each box are deterministic (for example P(s'|s,N)=1 if s' north of s). Note, however, that you cannot move outside the grid, thus all actions are not available in every box.
* When reaching **F**, the game ends (absorbing state).
* The numbers in the boxes represent the rewards you receive when moving into that box.
* Assume no discount in this model: $\gamma = 1$
    
    
| | | |
|----------|----------|---------|
|-1 |1|**F**|
|0|-1|1|  
|-1 |0|-1|  
|**S**|-1|1|

Let $(x,y)$ denote the position in the grid, such that $S=(0,0)$ and $F=(2,3)$.

**1a)** What is the optimal path of the MDP above? Is it unique? Submit the path as a single string of directions. For instance, NESW will make a circle.

The path that generates the best accumulation of rewards is EENNN. The total accumulated rewards will then be 0.

**1b)** What is the optimal policy (i.e., the optimal action in each state)? It is helpful if you draw the arrows/letters in the grid.

| | | | | |
|----------|----------|---------|--------|---------|
|-1|→|1|→|**F**|
|⇑||⇑||⇑|
|0|→|-1|→|1|  
|↓⇑||⇑||⇑|
|-1|→←|0|→|-1|  
|↓⇑||↓||⇑|
|**S**|→|-1|→←|1|

**1c)** What is expected total reward for the policy in 1a)?

\begin{align*}
V^\pi(s_1) &= \sum_{t=1}^5 r(s_t, a_t, s_{t+1})p(s_{t+1}|a_t, s_t) \\
&= \Big\{p(s_{t+1}|a_t, s_t) =
\begin{cases}
1 & \text{if } a_t \text{ adheres to the optimal path} \\
0 & \text{otherwise}
\end{cases}
\Big\} \\
&= r\Big((0,0), E, (1,0)\Big) + r\Big((1,0), E, (2,0)\Big) + r\Big((2,0), E, (2,1)\Big) + r\Big((2,1), E, (2,2)\Big) + r\Big((2,2), E, (2,3)\Big) \\
&= -1 + 1 - 1 + 1 + 0 \\
&= 0
\end{align*}


## Value Iteration

For larger problems we need to utilize algorithms to determine the optimal policy $\pi^*$. *Value iteration* is one such algorithm that iteratively computes the value for each state. Recall that for a policy to be optimal, it must satisfy the Bellman equation above, meaning that plugging in a given candidate $V^*$ in the right-hand side (RHS) of the Bellman equation should result in the same $V^*$ on the left-hand side (LHS). This property will form the basis of our algorithm. Essentially, it can be shown that repeated application of the RHS to any intial value function $V^0(s)$ will eventually lead to the value $V$ which statifies the Bellman equation. Hence repeated application of the Bellman equation will also lead to the optimal value function. We can then extract the optimal policy by simply noting what actions that satisfy the equation.    

The process of repeated application of the Bellman equation is what we here call the _value iteration_ algorithm. It practically procedes as follows:

```
epsilon is a small value, threshold
for x from i to infinity
do
    for each state s
    do
        V_k[s] = max_a Σ_s' p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
    end
    if  |V_k[s]-V_k-1[s]| < epsilon for all s
        for each state s,
        do
            π(s)=argmax_a ∑_s′ p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
            return π, V_k
        end
end

```

**Example:** We will illustrate the value iteration algorithm by going through two iterations. Below is a 3x3 grid with the rewards given in each state. Assume now that given a certain state $s$ and action $a$, there is a probability 0.8 that that action will be performed and a probability 0.2 that no action is taken. For instance, if we take action **E** in state $(x,y)$ we will go to $(x+1,y)$ 80 percent of the time (given that that action is available in that state), and remain still 20 percent of the time. We will use have a discount factor $\gamma = 0.9$. Let the initial value be $V^0(s)=0$ for all states $s\in S$.

**Reward**:

| | | |  
|----------|----------|---------|  
|0|0|0|
|0|10|0|  
|0|0|0|  


**Iteration 1**: The first iteration is trivial, $V^1(s)$ becomes the $\max_a \sum_{s'} p(s'|s,a) r(s,a,s')$ since $V^0$ was zero for all $s'$. The updated values for each state become

| | | |  
|----------|----------|---------|  
|0|8|0|
|8|2|8|  
|0|8|0|  
  
**Iteration 2**:  
  
Staring with cell (0,0) (lower left corner): We find the expected value of each move:  
Action **S**: 0  
Action **E**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **N**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **W**: 0

Hence any action between **E** and **N** would be best at this stage.

Similarly for cell (1,0):

Action **N**: 0.8( 10 + 0.9 \* 2) + 0.2(0 + 0.9 \* 8) = 10.88 (Action **N** is the maximizing action)  

Similar calculations for remaining cells give us:

| | | |  
|----------|----------|---------|  
|5.76|10.88|5.76|
|10.88|8.12|10.88|  
|5.76|10.88|5.76|  


## Question 2

**2a)** Code the value iteration algorithm just described here, and show the converging optimal value function and the optimal policy for the above 3x3 grid.








In [ ]:
# Value iteration algorithm

import numpy as np
import pandas as pd

eps = 0.1
p_stay = 0.2
p_leave = 1 - p_stay
discount = 0.9

# States / coordinates of the grid, counted from the bottom left corner
states = []
for j in range(0, 3):
    for i in range(0, 3):
        states.append((i,j))

# Reward matrix
rewards = np.zeros((3,3))
rewards[1][1] = 10
print(rewards)

# Expected reward matrix
V = np.zeros((3,3))


# Actions
actions = []
actions.append((0, 1)) # up
actions.append((0, -1)) # down
actions.append((-1, 0)) # left
actions.append((1, 0)) # right

change = 1

# Value iteration
while change > eps:
    change = 0
    old_V = np.copy(V)
    for state in states:
        maximum = float('-inf')
        for action in actions:
            new_state = tuple(map(sum, zip(state, action)))
            if new_state in states:
                expected_reward = p_leave * (rewards[new_state] + discount * old_V[new_state]) + p_stay * (rewards[state] + discount * old_V[state])
            else:
                expected_reward = p_stay * (rewards[state] + discount * old_V[state])
            maximum = max(maximum, expected_reward)
        V[state] = maximum
        change = max(change, abs(maximum - old_V[state]))
    print('\n\n', V)

# Calculate policy
policy = []
for state in states:
    maximum = float('-inf')
    best_actions = []
    for action in actions:
        new_state = tuple(map(sum, zip(state, action)))
        if new_state in states:
            expected_reward = p_leave * (rewards[new_state] + discount * V[new_state]) + p_stay * (rewards[state] + discount * V[state])
        else:
            expected_reward = p_stay * (rewards[state] + discount * V[state])
        if expected_reward > maximum:
            maximum = expected_reward
            best_actions = [action]
        elif expected_reward == maximum:
            best_actions.append(action)
    policy.append(best_actions)

for state in states:
    print('In state', state, 'take action(s)', policy[states.index(state)])

[[ 0.  0.  0.]
 [ 0. 10.  0.]
 [ 0.  0. 20.]]


 [[ 0.  8.  0.]
 [ 8.  2. 16.]
 [ 0. 16.  4.]]


 [[ 5.76 10.88 11.52]
 [10.88 13.88 21.76]
 [11.52 21.76 16.24]]


 [[ 8.8704 19.952  17.7408]
 [19.952  20.1656 31.6096]
 [17.7408 31.6096 22.5904]]


 [[15.962112 26.110592 25.952256]
 [26.110592 28.38872  37.954816]
 [25.952256 37.954816 30.825184]]


 [[21.6728064  33.13978496 31.9988736 ]
 [33.13978496 34.43743712 45.02599936]
 [31.9988736  45.02599936 36.87600064]]


 [[27.76175032 38.76011602 38.17851679]
 [38.76011602 40.61745822 50.65540035]
 [38.17851679 50.65540035 43.05639965]]


 [[32.90439859 44.2213908  43.34402127]
 [44.2213908  45.78303073 56.11857981]
 [43.34402127 56.11857981 48.22204019]]


 [[37.76219312 48.92363247 48.20730129]
 [48.92363247 50.646323   60.8212133 ]
 [48.20730129 60.8212133  53.0853447 ]]


 [[42.02221014 53.2716064  52.46858781]
 [53.2716064  54.90761172 65.16926658]
 [52.46858781 65.16926658 57.34663562]]


 [[45.91955443 57.12236959 56.36621774]
 [5


**2b)** Explain why the result of 2a) does not depend on the initial value $V_0$.


$$V_k(s) = \max_{a\in A} \left\{\sum_{s'\in S} p(s'|s,a)( r(s,a,s') +\gamma V_{k-1}(s')) \right\}$$

But we know that $V_{k-1}(s') depends on the previous step.

$$V_{k-1}(s') = \max_{a\in A} \left\{\sum_{s''\in S} p(s'|s',a)( r(s',a,s'') +\gamma V_{k-2}(s'')) \right\}$$

Put this into the original equation and we get:

$$V_k(s) = \max_{a\in A} \left\{\sum_{s'\in S} p(s'|s,a)( r(s,a,s') +\gamma \max_{a\in A} \left\{\sum_{s''\in S} p(s'|s',a)( r(s',a,s'') +\gamma V_{k-2}(s'')) \right\}) \right\}$$

We see that the term $V_{k-2}(s'')$ is multiplied with $\gamma^2$. If we continue working backwards like this, we will eventually hit the initial value $V_0$, multiplied with $\gamma^k$, which goes towards zero as $k$ gets larger since $0< \gamma < 1$. In other words, the initial value vanishes eventually. With a small epsilon, the 'change' between the iterations will only get sufficiently small after many iterations, and the initial value will make almost no contribution.

Note that having an initial $V_0$ close to satisfying the Bellmann equation will make the value iteration method converge with fewer iterations, and the initial value will still be a large part of the final answer.


**2c)** Describe your interpretation of the discount factor $\gamma$. What would happen in the two extreme cases $\gamma = 0$ and $\gamma = 1$? Given some MDP, what would be important things to consider when deciding on which value of $\gamma$ to use?

The discount factor decides what kind of time interval the agent should consider. For $\gamma = 0$ we'd get an agent that solely considers short-term rewards. So if the box looks like below and we start at s, it will still choose to go North instead of the road to East. North will be better short-term but much worse long term.

| | | |  
|----------|----------|---------|  
|-2|-2|F|
|10|-2|3|  
|s|7|2|

For $\gamma = 1$ the agent is as mentioned in the background theory segment. With no discount, all rewards along the way matter equally. In the example above, for $\gamma = 1$ would give us the road EENN (7 -> 2 -> 3 -> F).

Since the discount factor changes how far into the future the agent will consider rewards, the discount factor will be affecting the model heavily for uncertain and large environments. It will, for example, take much longer time for the agent to learn if it has to consider all of the future rewards ($\gamma = 1$) for an environment with many possible actions. Not only will a lower gamma in that case decrease the learning time, it will also decrease the agent's uncertainty on future actions. It's sort of deciding wether to exploit immediately or explore everything first. Explorations and exploitation will be further explained in question 4.

## Reinforcement Learning (RL) (Theory for optional question 3)
Until now, we understood that knowing the MDP, specifically $p(s'|a,s)$ and $r(s,a,s')$ allows us to efficiently find the optimal policy using the value iteration algorithm. Reinforcement learning (RL) or decision making under uncertainity, however, arises from the question of making optimal decisions without knowing the true world model (the MDP in this case).

So far we have defined the value function for a policy through $V^\pi$. Let's now define the *action-value function*

$$Q^\pi(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^\pi(s')]$$

The value function and the action-value function are directly related through

$$V^\pi (s) = \max_a Q^\pi (s,a)$$

i.e, the value of taking action $a$ in state $s$ and then following the policy $\pi$ onwards. Similarly to the value function, the optimal $Q$-value equation is:

$$Q^*(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^*(s')]$$

and the relationship between $Q^*(s,a)$ and $V^*(s)$ is simply

$$V^*(s) = \max_{a\in A} Q^*(s,a).$$

#### Q-learning

Q-learning is a RL-method where the agent learns about its unknown environment (i.e., the MDP is unknown) through exploration. In each time step *t* the agent chooses an action *a* based on the current state *s*, observes the reward *r* and the next state *s'*, and repeats the process in the new state. Q-learning is then a method that allows the agent to act optimally. Here we will focus on the simplest form of Q-learning algorithms, which can be applied when all states are known to the agent, and the state and action spaces are reasonably small. This simple algorithm uses a table of Q-values for each $(s,a)$ pair, which is then updated in each time step using the update rule in step $k+1$

$$Q_{k+1}(s,a) = Q_k(s,a) + \alpha \left( r(s,a) + \gamma \max \{Q_k(s',a')\} - Q_k(s,a) \right) $$

where $\gamma$ is the discount factor as before, and $\alpha$ is a pre-set learning rate. It can be shown that this algorithm converges to the optimal policy of the underlying MDP for certain values of $\alpha$ as long as there  is sufficient exploration. For our case, we set a constant $\alpha=0.1$.

#### OpenAI Gym

We shall use already available simulators for different environments (worlds) using the popular [OpenAI Gym library](https://www.gymlibrary.dev/). It just implements different types of simulators including ATARI games. Although here we will only focus on simple ones, such as the **Chain enviroment** illustrated below.
![alt text](https://chalmersuniversity.box.com/shared/static/6tthbzhpofq9gzlowhr3w8if0xvyxb2b.jpg)
The figure corresponds to an MDP with 5 states $S = \{1,2,3,4,5\}$ and two possible actions $A=\{a,b\}$ in each state. The arrows indicate the resulting transitions for each state-action pair, and the numbers correspond to the rewards for each transition.

## Question 3 (optional)
You are to first familiarize with the framework of [the OpenAI environments](https://www.gymlibrary.dev/), and then implement the Q-learning algorithm for the <code>NChain-v0</code> enviroment depicted above, using default parameters and a learning rate of $\gamma=0.95$. Report the final $Q^*$ table after convergence of the algorithm. For an example on how to do this, you can refer to the Q-learning of the **Frozen lake environment** (<code>q_learning_frozen_lake.ipynb</code>), uploaded on Canvas. *Hint*: start with a small learning rate.

Note that the NChain environment is not available among the standard environments, you need to load the <code>gym_toytext</code> package, in addition to the standard gym:

<code>
!pip install gym-legacy-toytext<br>
import gym<br>
import gym_toytext<br>
env = gym.make("NChain-v0")<br>
</code>

## Question 4

**4a)** What is the importance of exploration in reinforcement learning? Explain with an example.

Exploration is important to gain knowledge about new solutions or actions. The berry-example in the lessons was very good and we use this to explain the matter further. Except here we are a jam making buisness that wants our model to buy the best berries so we can make the best jam.

If the model is supposed to get the best berries and knows that ICA has berries it will go to ICA all the time if no other exploration is allowed. However, we have Willys and Coop too. We need to explore these stores to see if they have better berries than ICA, or else we might actually be buying the worst berries, the opposite of our goal! So, we allow the model to explore Willys and Coop sometimes. Perhaps the model learn that Willys has the best berries if we shop on a Tuesday, ICA on a Saturday and Coop’s is always bad. That way the model has explored and now knows new solutions/actions to berry-shopping.
However, to gain all this info we had to go to Coop several times to see if they might be better than the other stores all other days. That has created lots of days where to model has gone to Coop and gotten bad berries. If we had allowed all this exploration at Coop to be made in one month (explore all the time!!!), we would have gotten bad berries for an entire month. My berry-to-jam business would lose all its customers due to the bad quality ☹ And that’s why we allow the model to be exploiting as well! If it knows that ICA is selling perfect berries on Saturdays, we only allow the model to explore Coop a few Saturdays in a year, so that we won’t miss all the good berries. Also, only exploration a few days/weeks a month so we won’t be making bad jam for an entire month.

This can be translated into anything else. If we want to teach an AI to estimate passwords, we will start by giving it sentimental words commonly used in passwords from a database. For example, ‘Fido’ (dog name). But we will allow it to explore new combinations since it might find new commonly used keywords that we didn’t provide it with/didn’t know of in the first run. For example the word ‘summer’. That way it will teach itself to be more efficient.


Another example is learning to grocery shop for yourself. If you have learned that you can’t buy the most expensive cheese or your CSN will run out, do not be exploring and try a different cheese at the same expensive pricing just because an influencer told you to. Be exploiting and buy an ARLA cheese and then explore what sort of noodles are the best for you instead. That way we will eventually have the perfect grocery bag with products chosen by exploitation, but learned through exploration (I mean you had to buy that cheese in the first run to actually know it’s too expensive).  The shopping will go quick. Until the day a new cheese comes, that’s mid-priced. You now need to explore if the cheese is worth buying cheaper meatballs.

**4b)** Explain what makes reinforcement learning different from supervised learning tasks such as regression or classification.



Supervised learning tasks are trained on a labeled dataset to predict the output from new inputs. In contrast, reinforcement learning algorithms interact with an environment and are trained on a reward signal, rewards and punishments, which determine what actions are rewarded or punished. A supervised learning model map inputs to outputs, while a reinforcement model maps states to actions.

Let’s try to explain this by an example of disease classification.

A classification model gets trained on data that is labelled sick or healthy. A long data set of labels such as potential cancer birthmark, low risk birthmark, and so on. It can from that classify that all the cancer birthmark labelled data are the sick people that needs treatment. And by that recognize that all the red irregular birthmarks should be clustered with the other red irregular birthmarks. If a new picture gets clustered with this group, then you need treatment.

The reinforcement taught model will instead need to look at these birthmarks and trail-error by reinforcement. When it determines a birthmark correctly it will get positive reinforcement, and when it doesn’t it will get bad reinforcement. That way, it’ll be more likely to determine more red birthmarks as cancer, it will get some bad reinforcement on perfectly healthy red birthmarks and eventually learn that it’s the red, irregular birthmarks that needs to be alarmed as potential cancer.




## Question 5

**5a)** Give a summary of how a decision tree works and how it extends to random forests.

Decision tree is a supervised machine learning algorithm while random forest is a ensemble learning method.
Decision trees and random forests are used for regressions and classification based problem solving.
In decision tree, a tree-like structure is created by splitting the dataset into subsets for each node by considering the most important attributes.

At each node the decision making process takes place to identify the most significant attribute for the node selection.
Ex: For instance lets think of a scenario where we have to make a prediction on if a customer will make a purchase or not. For this the attributes the decision tree has to use are "the time spent by a customer for shopping" and "number of items purchased. But when the decision tree algorithm analyses the attributes it will choose "the time spent by a customer for shopping" as the most obvious attribute for the first split.

Upon the attribute selection, the dataset will be split into subsets to decide to which node the attribute will be fall into.
Ex: For the feature, "the time spent by a customer for shopping", the algorithm will determine a threshold of 40 minutes. Then the dataset will be divided into two subsets as "the customers shopping for 40 minutues or less" and "the customers shopping for more than 40 minutues".

The splitting process continous to the recursive subsets which gives rise to the tree structure. This process continues until there are no more conditions.
Ex: Lets assume the attibute "the customers shopping for 40 minutues or less" is selected. This can be further splitted as "items purchased by customers". This subset can be subjected to a threshold of 5 items.

The last step of decision tree creation is to create leaf nodes. This is where the final nodes or output values are displayed.
Ex: In the customer example, if a customer is shopping for 30 minutes and end up buying only 3 items, then its indicated as "no purchase" and if a customer is shopping for 30 minutes and buys 6 items then its indicated as "purchase".

In order to create a random forest from decision tree. The below steps should be followed.

The first step is to bootstrap. In bootstrapping, the multiple subsets of data are created instead of trainig one dataset.
Ex: Having a look at a dataset of 1000 shopping instances (customers). A Random Forest may construct three subsets (bootstrapped samples) of the data during the bootstrapped sampling phase, each having 800 instances. Here replacement sampling is used, certain instances may appear in many subsets, while others may be removed entirely.

Only a random subset of features are investigated for splitting at each node of every decision tree in the Random Forest.  
Ex: Let's assume there are features such as "Time Spent in Store," "Number of Items in Cart," and "Customer Age." During the splitting process, only two of these attributes are chosen, such as "Time Spent in the Store" and "Number of Items in the Cart," are considered for a certain tree.

Using bootstrapped subsets and random feature selection, multiple decision trees are trained independently.
Ex: Using bootstrapped subsets and random feature selection, the Random Forest constructs numerous decision trees separately. Assume it generates three decision trees, each trained on one of the bootstrapped samples and divided using a different random subset of features. Because this procedure can be parallelized for efficiency, it is appropriate for large datasets.

The Random Forest's ultimate forecast for classification tasks is established by a majority vote among the different trees. Each tree "votes" for a class, and the class that receives the most votes is the final forecast.
Ex: Each decision tree "votes" for a class in a classification problem (for example, predicting whether a client will make a purchase). Due to the overwhelming vote, if two trees predict "Purchase" and one predicts "No Purchase," the Random Forest's ultimate forecast will be "Purchase." Individual tree predictions are averaged for a regression task where the goal is to predict a numerical value (e.g., total purchase amount).


**5b)** State at least one advantage and one drawback with using random forests over decision trees. \\

Advantages
*   High generlizability, robustness and accurateness.
*   Overfitting is avoided.

Disadvantages

*   Hard to intepret.



# References
Primer/text based on the following references:
* http://www.cse.chalmers.se/~chrdimi/downloads/book.pdf
* https://github.com/olethrosdc/ml-society-science/blob/master/notes.pdf